# GenAISys Mobility
Adding a Generic Synthetic Trajectory Simulation and Predictive system to the book's GenAISys

📌 **Copyright 2025, Denis Rothman**  

This notebook integrates the *Trajectory simulation and Prediction* functionality built in 1_Trajectory_simulation_and_prediction.ipynb

The progam is implemented at three levels:  
**1.The IPython interface** under the name "mobility" at user level   
**2.The Handler Selection Mechanism** to include the mobility function in the handler registry      
**3.The AI Functions** to implement the trajectory simulation and prediction in the AI functions library.    

**Note:** You can choose to deactive DeepSeek or activate with the local installation(GPU required) or API options(CPU only required). In this notebook, DeepSeek was deactivated and only requires a CPU.


# ✅Setting up the environment

This notebook was developed in Google Colab. Colab includes many pre-installed libraries and sets `/content/` as the default directory, meaning you can access files directly by their filename if you wish (e.g., `filename` instead of needing to specify `/content/filename`). This differs from local environments, where you'll often need to install libraries or specify full file paths.

## Installing `click` and gTTS

**If you want to use text-to-speech in this notebook with `gTTS`, set `use_gtts` to `True` in the following cell.**

`click`, a command line library, is required for gTTS. This cell will set up the correct 'click' version required for gTTS and *display a restart message*. After restarting, simply click `Run All` to continue.

*The restart is necessary for Colab to take the new version into account.*



In [ ]:
use_gtts = False #activates Google TTS in Google Colab if True and deactivates if False

In [ ]:
# Cell 1: Conditional 'click' setup.
# This cell will only modify 'click' IF the 'click' version is not 8.1.8.
# If changes are made, a clear message will appear in the output, instructing you to restart the session.
# After restarting, Colab will reconnect, and you can simply "Run All" from the top.

import importlib.metadata
import os
import time

# Import for displaying rich output (HTML)
from IPython.display import display, HTML

#---Begin gTTS True or
current_click_version = None
try:
    current_click_version = importlib.metadata.version('click')
except importlib.metadata.PackageNotFoundError:
    pass

if not use_gtts:
    required_click_version = current_click_version
else:
    required_click_version = '8.1.8'
#---END gTTS True or False
print("--- Checking 'click' package version... ---")

try:
    current_click_version = importlib.metadata.version('click')
    print(f"Currently installed 'click' version: {current_click_version}")
except importlib.metadata.PackageNotFoundError:
    print("'click' package not found.")

# Check if current click version is not the required one
if current_click_version != required_click_version:
    print(f"\n--- 'click' version is not {required_click_version}. Initiating setup... ---")

    print("Uninstalling any existing 'click' installation...")
    !pip uninstall -y click

    print(f"Installing 'click=={required_click_version}' for compatibility...")
    !pip install click=={required_click_version}

    print("\n--- 'click' setup complete. ---")

    # Display a prominent, styled HTML message in the cell output
    html_message = """
    <div style="
        background-color: #e8f5e9; /* Very light green, calming */
        border: 2px solid #4caf50; /* Green border */
        padding: 15px;
        margin: 15px 0;
        border-radius: 8px;
        font-size: 1.1em; /* Slightly smaller font */
        color: #000000; /* Black text */
    ">
        <h3>Runtime Setup Complete</h3>
        <p>Package versions have been updated. For changes to take effect, you <strong>MUST restart the Colab runtime now.</strong></p>
        <p>Go to the menu: <strong><code>Runtime</code> &gt; <code>Restart runtime</code></strong>.</p>
        <p>After reconnecting, simply click <strong><code>Run All</code></strong> from the top to continue.</p>
    </div>
    """
    display(HTML(html_message))

    # Stop the Python cell execution gracefully.
    raise SystemExit("Please restart the Colab runtime to apply changes.")

else:
    print(f"--- 'click' is already at the correct version ({required_click_version}). No action needed. ---")

--- Checking 'click' package version... ---
Currently installed 'click' version: 8.2.1
--- 'click' is already at the correct version (8.2.1). No action needed. ---


## 🚀 **DeepSeek Activation Guide**  

#### ✅ **Option 1: Activate DeepSeek (`deepseek=True`)**  
🔹 **Resource Requirements**:  
- **GPU**: ~20GB VRAM (estimate)  
- **Disk Space**: 30-40GB  

🔹 **Setup Options**:  
- **On Google Colab**:  
  - **Recommended**: Google Colab **Pro** (with upgraded disk space).  
  - **Check**: Potential cost considerations.  
- **On a Local Machine**:  
  - **Recommended**: A **recent laptop** with a GPU.  
  - **No additional cost** required.  

---

#### ❌ **Option 2: No DeepSeek Activation (`deepseek=False`)**  
🔹 **Resource Requirements**:  
- **No GPU** required (CPU is sufficient).  
- **No additional disk space needed**.  
- **No cost** for local execution.  

🔹 **Limitations**:  
- DeepSeek **won’t be installed** in this notebook.  
- The notebook will default to using the **OpenAI framework** (which requires an OpenAI API token and incurs costs).  
- Alternatively, you can use the **DeepSeek API** via `pip install openai`, but API calls will be charged based on DeepSeek’s pricing.  

---

💡 **Key Takeaway**  
This setup gives you flexibility to explore different execution environments and choose the best fit for your project! 🚀


In [ ]:
# DeepSeek activation deepseek=True to activate. 20 Go (estimate) GPU memory and 30-40 Go Disk Space
deepseek=False

In [ ]:
if deepseek==True:
  from google.colab import drive
  drive.mount('/content/drive')

## File downloading script

grequests contains a script to download files from the repository

In [ ]:
!curl -L https://raw.githubusercontent.com/uCertifyLLC/Building-Business-Ready-Generative-AI-Systems/master/commons/grequests.py --output grequests.py

## Setting up the DeepSeek Hugging Face environment

### Checking GPU activation

In [ ]:
if deepseek==True:
  !nvidia-smi

### Activate cache in Google Drive

In [ ]:
import os

if deepseek==True:
  # Define the cache directory in your Google Drive
  cache_dir = '/content/drive/MyDrive/genaisys/HuggingFaceCache'

  # Set environment variables to direct Hugging Face to use this cache directory
  os.environ['TRANSFORMERS_CACHE'] = cache_dir
  #os.environ['HF_DATASETS_CACHE'] = os.path.join(cache_dir, 'datasets')

### Installation Hugging Face environment

Path in this notebook: drive/MyDrive/genaisys/


In [ ]:
if deepseek==True:
  !pip install transformers

### Checking transformer version

In [ ]:
if deepseek==True:
  import transformers
  print(transformers.__version__)

### Model

In [ ]:
import time
if deepseek==True:
  from transformers import AutoTokenizer, AutoModelForCausalLM

  # Define the path to the model directory
  model_path = '/content/drive/MyDrive/genaisys/HuggingFaceCache/models--unsloth--DeepSeek-R1-Distill-Llama-8B/snapshots/71f34f954141d22ccdad72a2e3927dddf702c9de'

  # Record the start time
  start_time = time.time()
  # Load the tokenizer and model from the specified path
  tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
  model = AutoModelForCausalLM.from_pretrained(model_path, device_map='auto', torch_dtype='auto', local_files_only=True)

  # Record the end time
  end_time = time.time()

  # Calculate the elapsed time
  elapsed_time = end_time - start_time

  print(f"Time taken to load the model: {elapsed_time:.2f} seconds")


In [ ]:
# Retrieve the Hugging Face token from Colab's Secrets Manager: uncomment only if requested
#if deepseek==True:
  #from google.colab import userdata
  #userdata.get('HF_TOKEN')

## OpenAI

In [ ]:
from grequests import download
download("commons","requirements01.py")
download("commons","openai_setup.py")
download("commons","reason.py")
download("commons","machine_learning.py")

### Installing OpenAI

In [ ]:
# Run the setup script to install and import dependencies
%run requirements01

#### Initializing the OpenAI API key



In [ ]:
google_secrets=True #activates Google secrets in Google Colab
if google_secrets==True:
  import openai_setup
  openai_setup.initialize_openai_api()

In [ ]:
if google_secrets==False: # Uncomment the code and choose any method you wish to initialize the API_KEY
  import os
  #API_KEY=[YOUR API_KEY]
  #os.environ['OPENAI_API_KEY'] = API_KEY
  #openai.api_key = os.getenv("OPENAI_API_KEY")
  #print("OpenAI API key initialized successfully.")

#### Importing the API call function

In [ ]:
# Import the function from the custom OpenAI API file
import os
import reason
from reason import make_openai_api_call
from reason import make_openai_reasoning_call

## Installing gtts

gTTS (Google Text-to-Speech) is a Python library and CLI tool that interfaces with Google Translate's text-to-speech API. It allows users to convert text into spoken words, supporting multiple languages and accents, and can save the output as MP3 files.  

In [ ]:
# use_gtts activates Google TTS in Google Colab if use_gtts is True and deactivates if False
if use_gtts:
 !pip install gTTS==2.5.4
 from gtts import gTTS
 from IPython.display import Audio

def text_to_speech(text):
    # Convert text to speech and save as an MP3 file
    if use_gtts:
      if not isinstance(text, str):
            text = str(text) # Making sure the text is a string not a list
      tts = gTTS(text)
      tts.save("response.mp3")

## Machine learning

In [ ]:
# Import the function from the custom OpenAI API file
import os
import machine_learning
from machine_learning import ml_agent

## Chain of Thought(COT)

In [ ]:
# Import the function from the custom OpenAI API file
import os
import reason
from reason import chain_of_thought_reasoning
from reason import memory_reasoning_thread # import memory reasoning thread96

In [ ]:
# AI agent : the messages and prompts for memory agent tasks
download("commons","cot_messages_c6.py") # downloaded messages and prompts

## Installing Pinecone

In [ ]:
download("commons","requirements02.py")

In [ ]:
# Run the setup script to install and import dependencies
%run requirements02

### Initializing the Pinecone API key

In [ ]:
download("commons","pinecone_setup.py")

In [ ]:
if google_secrets==True:
  import pinecone_setup
  pinecone_setup.initialize_pinecone_api()

In [ ]:
if google_secrets==False: # Uncomment the code and choose any method you wish to initialize the Pinecone API key
  import os
  #PINECONE_API_KEY=[YOUR PINECONE_API_KEY]
  #os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
  #openai.api_key = os.getenv("PINECONE_API_KEY")
  #print("OpenAI API key initialized successfully.")

##  The Pinecone index

In [ ]:
import os
from pinecone import Pinecone, ServerlessSpec
# Retrieve the API key from environment variables
api_key = os.environ.get('PINECONE_API_KEY')
if not api_key:
    raise ValueError("PINECONE_API_KEY is not set in the environment!")

# Initialize the Pinecone client
pc = Pinecone(api_key=api_key)

In [ ]:
from pinecone import ServerlessSpec

index_name = 'genai-v1'
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
import time
import pinecone
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimension of the embedding model
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

## Querying functions

In [ ]:
def display_results(query_results):
  for match in query_results['matches']:
    print(f"ID: {match['id']}, Score: {match['score']}")
    if 'metadata' in match and 'text' in match['metadata']:
        text=match['metadata']['text']
        #print(f"Text: {match['metadata']['text']}")
        target_id = query_results['matches'][0]['id']  # Get the ID from the first match
                #print(f"Target ID: {target_id}")
    else:
        print("No metadata available.")
  return text, target_id


In [ ]:
import openai
client = openai.OpenAI()
embedding_model = "text-embedding-3-small"
def get_embedding(text, model=embedding_model):
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    embedding = response.data[0].embedding
    return embedding

In [ ]:
def get_query_results(query_text, namespace):
    # Generate the query vector from the query text
    query_vector = get_embedding(query_text)  # Replace with your method to generate embeddings

    # Perform the query
    query_results = index.query(
        vector=query_vector,
        namespace=namespace,
        top_k=1,  # Adjust as needed
        include_metadata=True
    )
    # Return the results
    return query_results

In [ ]:
def query_vector_store(query_text, namespace):
    print("Querying vector store...")

    # Retrieve query results
    query_results = get_query_results(query_text, namespace)

    # Process and display the results
    print("Processed query results:")
    text, target_id = display_results(query_results)

    return text, target_id

# ✅AI Agent

## Generic Synthetic Trajectory Simulation and Predictive System

### Generic Synthetic Trajectory Simulation

In [ ]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from PIL import Image as PILImage

def create_grid_with_trajectory(grid_size=200, num_points=50, missing_count=5):
    grid = np.zeros((grid_size, grid_size), dtype=int)
    trajectory = []

    x = random.randint(0, grid_size - 1)
    y = random.randint(0, grid_size - 1)
    day = random.randint(1, 365)
    timeslot = random.randint(0, 47)

    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    current_dir_index = random.randint(0, 3)

    turn_weights = {-1: 0.15, 0: 0.70, 1: 0.15}

    for _ in range(num_points):
        turn = random.choices(list(turn_weights.keys()), weights=list(turn_weights.values()))[0]
        current_dir_index = (current_dir_index + turn) % len(directions)
        dx, dy = directions[current_dir_index]
        new_x = x + dx
        new_y = y + dy

        if not (0 <= new_x < grid_size and 0 <= new_y < grid_size):
            valid_indices = [idx for idx, (dx_temp, dy_temp) in enumerate(directions) if 0 <= x + dx_temp < grid_size and 0 <= y + dy_temp < grid_size]
            if valid_indices:
                current_dir_index = random.choice(valid_indices)
                dx, dy = directions[current_dir_index]
                new_x = x + dx
                new_y = y + dy
            else:
                new_x, new_y = x, y

        x, y = new_x, new_y
        trajectory.append((day, timeslot, x, y))
        grid[x, y] = 1
        timeslot = (timeslot + random.randint(1, 3)) % 48

    missing_indices = random.sample(range(len(trajectory)), min(missing_count, len(trajectory)))
    for idx in missing_indices:
        d, t, _, _ = trajectory[idx]
        trajectory[idx] = (d, t, 999, 999)

    x_coords = [x if x != 999 else np.nan for _, _, x, y in trajectory]
    y_coords = [y if y != 999 else np.nan for _, _, x, y in trajectory]

    plt.figure(figsize=(8, 8))
    plt.plot(x_coords, y_coords, marker='o', linestyle='-', color='blue', label="Agent Trajectory")

    valid_indices = [i for i, (xx, yy) in enumerate(zip(x_coords, y_coords)) if not (np.isnan(xx) or np.isnan(yy))]
    if len(valid_indices) > 1:
        valid_x = [x_coords[i] for i in valid_indices]
        valid_y = [y_coords[i] for i in valid_indices]
        dx = np.diff(valid_x)
        dy = np.diff(valid_y)
        plt.quiver(valid_x[:-1], valid_y[:-1], dx, dy, angles='xy', scale_units='xy', scale=1, color='red', width=0.005, label="Direction")

    for (xx, yy) in zip(x_coords, y_coords):
        if not np.isnan(xx) and not np.isnan(yy):
            plt.plot(xx, yy, marker='D', markersize=10, color='green', label='Start')
            break

    for i, (d, t, x, y) in enumerate(trajectory):
        if x == 999 and y == 999:
            prev_valid = next_valid = None
            for j in range(i - 1, -1, -1):
                _, _, xp, yp = trajectory[j]
                if xp != 999 and yp != 999:
                    prev_valid = (xp, yp)
                    break
            for j in range(i + 1, len(trajectory)):
                _, _, xp, yp = trajectory[j]
                if xp != 999 and yp != 999:
                    next_valid = (xp, yp)
                    break
            marker_x, marker_y = prev_valid if prev_valid else next_valid if next_valid else (None, None)
            if marker_x is not None:
                plt.plot(marker_x, marker_y, marker='x', markersize=10, color='magenta', label='Missing Data' if i == missing_indices[0] else "")

    plt.title("Agent Trajectory with Direction Arrows and Missing Data")
    plt.xlabel("X coordinate")
    plt.ylabel("Y coordinate")
    plt.grid(True)
    plt.legend()
    plt.savefig("mobility.png")
    plt.close()

    return grid, trajectory

### Generative Predictive Engine

In [ ]:
import json
import os
import textwrap
from PIL import Image as PILImage

def handle_mobility_orchestrator(muser_message1, msystem_message_s1, mgeneration, mimcontent4, mimcontent4b):
    grid, trajectory = create_grid_with_trajectory(grid_size=200, num_points=50, missing_count=5)

    trajectory_json = json.dumps({"trajectory": trajectory}, indent=2)
    #print("Trajectory Data (JSON):\n", trajectory_json)
    muser_message = f"{muser_message1}\n\nHere is the trajectory data:\n{trajectory_json}"

    reasoning_steps = reason.mobility_agent_reasoning_thread(
        muser_message, msystem_message_s1, mgeneration, mimcontent4, mimcontent4b
    )

    reasoning_steps.insert(0, ("Generated Trajectory Data:", trajectory))

    return reasoning_steps

##AI Functions

In [ ]:
import openai
from openai import OpenAI
from IPython.display import display, Image
import requests

# Initialize the OpenAI client
client = OpenAI()

# Global variable to ensure memory is always used
memory_enabled = True  # Set to True to retain conversation memory

# AI agent: Download messages and prompts
download("commons", "cot_messages_c6.py")  # Downloaded messages and prompts

# Define Handler Functions
def handle_pinecone_rag(user_message, **kwargs):
    if "Pinecone" in user_message:
      namespace = "genaisys"
    if "RAG" in user_message:
      namespace = "data01"

    print(namespace)

    query_text = user_message
    query_results = get_query_results(query_text, namespace)

    print("Processed query results:")
    qtext, target_id = display_results(query_results)
    print(qtext)

    # Run task
    sc_input = qtext + " " + user_message

    models = kwargs.get("models", "OpenAI")  # Default to OpenAI if not provided
    if models == "DeepSeek" and deepseek==False:
       models="OpenAI"


    if models == "OpenAI":
      task_response = reason.make_openai_api_call(
      sc_input, "system","You are an assistant who executes the tasks you are asked to do.", "user")

    if models == "DeepSeek":
      # Tokenize the input
      inputs = tokenizer(sc_input, return_tensors='pt').to('cuda')
      # Generate output
      outputs = model.generate(**inputs, max_new_tokens=1200)
      # Decode the output
      task_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return f"{namespace}:{models}: {task_response}"

def handle_reasoning_customer(user_message, **kwargs):
    initial_query = user_message
    download("Chapter05", "customer_activities.csv")

    reasoning_steps = reason.chain_of_thought_reasoning(initial_query)
    return reasoning_steps

def handle_analysis(user_message, **kwargs):
    from cot_messages_c6 import system_message_s1

    models = kwargs.get("models", "OpenAI")  # Default to OpenAI if not provided
    if models == "DeepSeek" and deepseek==False:
      models="OpenAI"

    if models == "OpenAI":
      reasoning_steps = reason.make_openai_reasoning_call(user_message, system_message_s1)

    if models == "DeepSeek":
      # Tokenize the input
      ds_input=system_message_s1+user_message
      inputs = tokenizer(ds_input, return_tensors='pt').to('cuda')
      # Generate output
      outputs = model.generate(**inputs, max_new_tokens=1200)
      # Decode the output
      reasoning_steps = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return reasoning_steps

def handle_generation(user_message, **kwargs):
    from cot_messages_c6 import system_message_s1, generation, imcontent4, imcontent4b
    reasoning_steps = reason.memory_reasoning_thread(user_message, system_message_s1, generation, imcontent4, imcontent4b)
    return reasoning_steps

def handle_mobility(user_message, **kwargs):
    from cot_messages_c6 import msystem_message_s1, mgeneration, mimcontent4,muser_message1
    mimcontent4b=mimcontent4
    #call Generic Synthetic Trajectory Simulation and Predictive System
    reasoning_steps = handle_mobility_orchestrator(muser_message1, msystem_message_s1, mgeneration, mimcontent4, mimcontent4b)
    return reasoning_steps


def handle_image_creation(user_message, **kwargs):
    prompt = user_message
    image_url = reason.generate_image(prompt, model="dall-e-3", size="1024x1024", quality="standard", n=1)

    # Save the image locally
    save_path = "c_image.png"
    image_data = requests.get(image_url).content
    with open(save_path, "wb") as file:
        file.write(image_data)

    return "Image created"

'''
def handle_no_memory(user_message, **kwargs):
    task_response = reason.make_openai_api_call(
        user_message, "system",
        "You are an assistant who executes the tasks you are asked to do.", "user"
    )
    return task_response
'''
def handle_with_memory(messages, user_message, **kwargs):
    global memory_enabled  # Ensure global memory setting is used

    # If memory is disabled, respond with a message
    if not memory_enabled:
        return "Memory is disabled."

    # Extract all past messages (user + assistant) from the conversation history
    conversation_history = [
        f"{msg['role'].capitalize()}: {msg['content']}"
        for msg in messages if "content" in msg
    ]

    # Combine all conversation history
    combined_history = "\n".join(conversation_history)

    # Append the latest user message to the history
    full_context = f"{combined_history}\nUser: {user_message}"

    models = kwargs.get("models", "OpenAI")  # Default to OpenAI if not provided

    if models == "OpenAI":
        task_response = reason.make_openai_api_call(
            full_context, "system",
            "You are an assistant who executes the tasks you are asked to do.", "user"
        )

    elif models == "DeepSeek":
        # Tokenize the full conversation history for DeepSeek
        sys_prompt = "You are an assistant who executes the tasks you are asked to do."
        ds_input = f"{sys_prompt}\n{full_context}"
        inputs = tokenizer(ds_input, return_tensors='pt').to('cuda')

        # Generate output
        outputs = model.generate(**inputs, max_new_tokens=1200)

        # Decode the output
        task_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Store bot response in memory
    messages.append({"role": "assistant", "content": task_response})

    return task_response

## Handler registry

In [ ]:
# Handler Registry
handlers = [
    # Pinecone / RAG handler: check only the current user message
    (
        lambda msg, instruct, mem, models, user_message, **kwargs: "Pinecone" in user_message or "RAG" in user_message,
        lambda msg, instruct, mem, models, user_message, **kwargs: handle_pinecone_rag(user_message, models=models)
    ),

    # Reasoning handler: check only the current user message
    (
        lambda msg, instruct, mem, models, user_message, **kwargs: all(keyword in user_message for keyword in ["Use reasoning", "customer", "activities"]),
        lambda msg, instruct, mem, models, user_message, **kwargs: handle_reasoning_customer(user_message, models=models)
    ),

    # Analysis handler: determined by the instruct flag
    (
        lambda msg, instruct, mem, models, user_message, **kwargs: instruct == "Analysis",
        lambda msg, instruct, mem, models, user_message, **kwargs: handle_analysis(user_message, models=models)
    ),

    # Generation handler: determined by the instruct flag
    (
        lambda msg, instruct, mem, models, user_message, **kwargs: instruct == "Generation",
        lambda msg, instruct, mem, models, user_message, **kwargs: handle_generation(user_message, models=models)
    ),

    # Mobility handler: determined by the instruct flag
    (
        lambda msg, instruct, mem, models, user_message, **kwargs: instruct == "Mobility",
        lambda msg, instruct, mem, models, user_message, **kwargs: handle_mobility(user_message, models=models)
    ),

    # Create image handler: check only the current user message
    (
        lambda msg, instruct, mem, models, user_message, **kwargs: "Create" in user_message and "image" in user_message,
        lambda msg, instruct, mem, models, user_message, **kwargs: handle_image_creation(user_message, models=models)
    )
]

# Append the fallback memory handler for when instruct is "None"
handlers.append(
    (
        lambda msg, instruct, mem, models, user_message, **kwargs: instruct == "None",
        lambda msg, instruct, mem, models, user_message, **kwargs: handle_with_memory(
            msg, user_message,
            files_status=kwargs.get('files_status'),
            instruct=instruct,
            mem=memory_enabled,  # ✅ Replace user_memory with memory_enabled
            models=models
        )
    )
)




## Handler selection mechanism

In [ ]:
def chat_with_gpt(messages, user_message, files_status, active_instruct, models):
    global memory_enabled  # Ensure memory is used if set globally

    try:
        # Iterate over handlers and execute the first matching one
        for condition, handler in handlers:
            if condition(messages, active_instruct, memory_enabled, models, user_message):
                return handler(messages, active_instruct, memory_enabled, models, user_message, files_status=files_status)

        # If no handler matched, default to memory handling with full conversation history
        return handle_with_memory(
            messages,  # ✅ Now passing full message history
            user_message,
            files_status=files_status,
            instruct=active_instruct,
            mem=memory_enabled,  # ✅ Ensuring memory usage
            models=models
        )
    except Exception as e:
        return f"An error occurred in the handler selection mechanism: {str(e)}"


# ✅GenAISys IPython interface

## Processing text

In [ ]:
def format_json_as_markdown(data, level=0):
    """Format JSON-like data as Markdown with proper indentation."""
    html_output = ""
    indent = "  " * level

    if isinstance(data, dict):
        for key, value in data.items():
            html_output += f"{indent}**{key}**:<br>\n"
            html_output += format_json_as_markdown(value, level + 1)
    elif isinstance(data, list):
        for item in data:
            html_output += format_json_as_markdown(item, level)
    else:
        html_output += f"{indent}{data}<br>\n"

    return html_output or ""  # Ensure a string is always returned

In [ ]:
def format_entry(entry):
    """Format the content of an entry for Markdown display."""
    if entry['role'] == 'user':
        formatted_content = format_json_as_markdown(entry['content']) if isinstance(entry['content'], (dict, list)) else entry['content']
        formatted_content = formatted_content.replace("\n", "<br>")  # Process newlines outside the f-string
        return f"**<span style='color: blue;'>{active_user}:</span>** {formatted_content}"
    elif entry['role'] == 'assistant':
        formatted_content = format_json_as_markdown(entry['content']) if isinstance(entry['content'], (dict, list)) else entry['content']
        formatted_content = formatted_content.replace("\n", "<br>")  # Process newlines outside the f-string
        return f"**<span style='color: green;'>Agent:</span>** {formatted_content}"
    else:
        return entry['content']  # Fallback for unrecognized roles

## 🚀Running the interface

In [ ]:
# Import required modules
from IPython.display import display, HTML, clear_output, Markdown
from ipywidgets import Dropdown, Textarea, Button, Checkbox, VBox, Layout, Output
from PIL import Image as PILImage
import json
import os

# Create an output widget for reasoning steps
reasoning_output = Output(layout=Layout(border="1px solid black", padding="10px", margin="10px", width="100%"))

# Initialize conversation histories for all users and active user
user_histories = {"User01": [], "User02": [], "User03": []}
active_user = "User01"  # Default user
conversation_active = True

# Function to handle user input and optional bot response
def chat(user_message):
    global conversation_active
    if user_message.lower() in ['exit', 'quit']:
        conversation_active = False
        clear_output(wait=True)
        display(HTML("<div style='color: red;'><strong>Conversation ended. Saving history...</strong></div>"))
        save_conversation_history()
        display(HTML("<div style='color: green;'><strong>History saved. Proceed to the next cell.</strong></div>"))
        return

    # Append user message to active user's history
    user_histories[active_user].append({"role": "user", "content": user_message})

    # Generate bot response if agent_checkbox is checked
    if agent_checkbox.value:
        pfiles = 1 if files_checkbox.value else 0
        active_instruct = instruct_selector.value
        selected_model = model_selector.value
        response = chat_with_gpt(user_histories[active_user], user_message, pfiles, active_instruct, models=selected_model)

        # Append bot response to active user's history
        user_histories[active_user].append({"role": "assistant", "content": response})

        # If TTS is enabled, convert response to speech
        if tts_checkbox.value:
          if isinstance(response, list):
              response = " ".join(response)  # Convert list to string if necessary
          text_to_speech(response)

    # Update display
    update_display()

# Function to update the display
def update_display():
    clear_output(wait=True)

    for entry in user_histories[active_user]:
        formatted_entry = format_entry(entry)
        display(Markdown(formatted_entry))

    #Audio display
    if os.path.exists("/content/response.mp3"):
      display(Audio("/content/response.mp3", autoplay=True))
      !rm /content/response.mp3


    # Check if Files are enabled
    if files_checkbox.value == True:
        # Display c_image.png if it exists
        if os.path.exists("c_image.png"):
            original_image = PILImage.open("c_image.png")
            new_size = (original_image.width // 2, original_image.height // 2)
            resized_image = original_image.resize(new_size)
            display(resized_image)

        # Display mobility.png if it exists and the "Mobility" instruction is selected
        if os.path.exists("mobility.png") and instruct_selector.value == "Mobility":
            original_image = PILImage.open("mobility.png")
            display(original_image)

    # Display interactive widgets
    if conversation_active:
        display(
            VBox(
                [user_selector, input_box, submit_button, agent_checkbox, tts_checkbox, files_checkbox, instruct_selector, model_selector],
                layout=Layout(display='flex', flex_flow='column', align_items='flex-start', width='100%')
            )
        )

    # Display reasoning_output persistently
    display(reasoning_output)

# Function to handle submission (button click or Enter key)
def handle_submission():
    user_message = input_box.value.strip()  # Get input text
    if user_message:
        input_box.value = ""  # Clear input box

        # Show "Processing request..." immediately
        with reasoning_output:
            reasoning_output.clear_output(wait=True)
            print("Processing request...")

        # Check if instruct_selector is "Analysis" or "Generation"
        if instruct_selector.value in ["Analysis", "Generation","Mobility"]:
            with reasoning_output:
                reasoning_output.clear_output(wait=True)
                print("Thinking...")

        # Process user message
        chat(user_message)

        # Indicate that processing is finished
        with reasoning_output:
            reasoning_output.clear_output(wait=True)
            print("Process completed.")

# Function to handle submit button click
def handle_button_click(sender):
    handle_submission()

# Function to handle Enter key press in the Textarea
def handle_enter_key(change):
    if change['new'].endswith("\n"):  # Detect Enter key press
        handle_submission()

# Function to update active user
def on_user_change(change):
    global active_user
    active_user = change['new']
    update_display()

# Function to save conversation history to a file
def save_conversation_history():
    filename = "conversation_history.json"
    with open(filename, 'w') as file:
        json.dump(user_histories, file, indent=4)
    display(HTML(f"<div style='color: green;'><strong>Conversation history saved to {filename}.</strong></div>"))

# Create dropdown for user selection
user_selector = Dropdown(
    options=["User01", "User02", "User03"],
    value=active_user,
    description='User:',
    layout=Layout(width='50%')
)
user_selector.observe(on_user_change, names='value')

# Create multi-line input box
input_box = Textarea(
    placeholder="Type your message here or type 'exit' or 'quit' to end the conversation.",
    layout=Layout(width='100%', height='100px')
)

# Create submit button
submit_button = Button(description="Send", button_style='primary')
submit_button.on_click(handle_button_click)

# Attach event handler for Enter key (FIXED)
input_box.observe(handle_enter_key, names="value")

# Create checkboxes for toggles
tts_checkbox = Checkbox(value=False, description='Voice Output', layout=Layout(width='20%'))
files_checkbox = Checkbox(value=False, description='Files', layout=Layout(width='20%'))
agent_checkbox = Checkbox(value=True, description='Agent', layout=Layout(width='20%'))


def on_files_checkbox_change(change):
    # Only remove images if the checkbox changed from True to False.
    if change['old'] == True and change['new'] == False:
        if os.path.exists("c_image.png"):
            os.remove("c_image.png")
        if os.path.exists("mobility.png"):
            os.remove("mobility.png")

# Attach the observer to files_checkbox
files_checkbox.observe(on_files_checkbox_change, names='value')

# Function to update instruct selector
def on_instruct_change(change):
    global active_instruct
    active_instruct = change['new']
    update_display()

# Dropdown for reasoning type
instruct_selector = Dropdown(
    options=["None", "Analysis", "Generation","Mobility"],
    value="None",
    description='Reasoning:',
    layout=Layout(width='50%')
)
instruct_selector.observe(on_instruct_change, names='value')

# Dropdown for model selection
model_selector = Dropdown(
    options=["OpenAI", "DeepSeek"],
    value="OpenAI",
    description="Model:",
    layout=Layout(width="50%")
)

# Display interactive widgets
display(
    VBox(
        [user_selector, input_box, submit_button, agent_checkbox, tts_checkbox, files_checkbox, instruct_selector, model_selector],
        layout=Layout(display='flex', flex_flow='column', align_items='flex-start', width='100%')
    )
)

# Display reasoning output
with reasoning_output:
    reasoning_output.clear_output(wait=True)
    print("Reasoning activated")

# Load and display the conversation history

**Note:** *Make sure you have had a conversation by running the interface in the preceding cell. Then make sure you stopped the conversion with 'quit' or 'exit" so that the conversation is saved. If not, the conversation will not be displayed and summarized in the following cells.*


In [ ]:
import json
from IPython.display import display, Markdown
import os

display_conversation_history=True
summary=True

if display_conversation_history == True or summary==True:
    # File path
    file_path = 'conversation_history.json'

    # Check if the file exists
    if os.path.exists(file_path):
        print(f"The file '{file_path}' exists.")
    else:
        print(f"The file '{file_path}' does not exist.")
        display_conversation_history=False
        summary=False
        print("Conversation history not processed")

In [ ]:
# Display option
if display_conversation_history==True:
  # File path
  file_path = 'conversation_history.json'

  # Open the file and read its content into the 'dialog' variable
  with open(file_path, 'r', encoding='utf-8') as file:
      dialog = json.load(file)  # Parse JSON content

  # Function to format JSON content as markdown
  def format_json_as_markdown(data, level=0):
      html_output = ""
      indent = "  " * level
      if isinstance(data, dict):
          for key, value in data.items():
              html_output += f"{indent}**{key}**:<br>\n"
              html_output += format_json_as_markdown(value, level + 1)
      elif isinstance(data, list):
          for item in data:
              html_output += format_json_as_markdown(item, level)
      else:
          html_output += f"{indent}{data}<br>\n"
      return html_output

  # Format the JSON into markdown
  formatted_markdown = format_json_as_markdown(dialog)

  # Display formatted JSON as Markdown
  display(Markdown(formatted_markdown))

# Load and summarize the conversation history

In [ ]:
import json
from IPython.display import Markdown, display

def summarize_conversation(file_path):
    """
    Reads a conversation history JSON file, formats it, and generates a detailed
    summary with a list of actions from the JSON text. The summary is displayed in Markdown.

    Parameters:
        file_path (str): Path to the JSON file containing conversation history.

    Returns:
        None: The summary is displayed as Markdown output.
    """
    # Step 1: Read the conversation history from the JSON file
    with open(file_path, 'r', encoding='utf-8') as file:
        dialog = file.read()
    conversation_history_json = json.loads(dialog)

    # Step 2: Construct dialog string from the JSON conversation history
    def construct_dialog(conversation_history_json):
        dialog = ""
        for user, messages in conversation_history_json.items():
            dialog += f"\n{user}:\n"
            for message in messages:
                role = message["role"]
                content = message["content"]
                dialog += f"- {role}: {content}\n"
        return dialog

    formatted_dialog = construct_dialog(conversation_history_json)

    # Step 3: Prepare the task for the summary
    mrole = "system"
    mcontent = "Your task is to read this JSON formatted text and summarize it."
    user_role = "user"
    task = f"Read this JSON formatted text and make a very detailed summary of it with a list of actions:\n{formatted_dialog}"

    # Step 4: Call the `make_openai_api_call` function
    task_response = make_openai_api_call(task, mrole, mcontent, user_role)

    # Step 5: Display the task response as Markdown
    display(Markdown(task_response))


In [ ]:
if summary==True:
    # File path to the JSON file
    file_path = '/content/conversation_history.json'

    # Check if the file exists before calling the function
    if os.path.exists(file_path):
        summarize_conversation(file_path)
    else:
        print(f"File '{file_path}' does not exist. Please provide a valid file path.")
